In [ ]:
%load_ext autoreload
%autoreload 2
%config InteractiveShell.ast_node_interactivity = 'all'
%config Completer.use_jedi = False

In [ ]:
%%html
<style>
@import url('https://fonts.cdnfonts.com/css/fantasque-sans-mono');
@import url('https://fonts.googleapis.com/css2?family=Victor+Mono:ital,wght@0,100..700;1,100..700&display=swap');
/* Any CSS style can go in here. */
.dataframe th {
  font-size: 11px;
  font-weight: 700;
  font-style: italic;
  font-family: 'Victor Mono', monospace;
}

.dataframe td {
  font-size: 10px;
  font-weight: 400;
  font-family: 'Victor Mono', monospace;
}

.output_html pre {
  font-family: 'Victor Mono', monospace !important;
}
</style>


In [ ]:
# **Package Imports**
# - From the Python Standard Library
import logging
import os
import sys

# - From third party libraries
import api_24sea
import dotenv
import matplotlib.pyplot as plt
import pandas as pd
from api_24sea.datasignals import fatigue


In [ ]:
# **Package Versions**
print(f"📂 \033[1m\033[93mWorking Folder:     \033[0;34;4m{os.getcwd()}\033[0m")
print(f"🐼 \033[1m\033[93mPandas Version:     \033[0;3m{pd.__version__}\033[0m")
print(f"⚡  \033[1m\033[93mSwifter Version:    \033[0;3m{fatigue.S.swifter.__version__}\033[0m")
print(f"💪 \033[1m\033[93mPy-fatigue Version: \033[0;3m{fatigue.S.py_fatigue.__version__}\033[0m")
print(f"🌊 \033[1m\033[93mApi-24SEA Version:  \033[0;3m{api_24sea.__version__}\033[0m")
print(f"🐍 \033[1m\033[93mPython Version:     \033[0;3m{sys.version}\033[0m")
# **Notebook Configuration**
logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [ ]:
# **Load Environment Variables from .env File**
_ = dotenv.load_dotenv(dotenv.find_dotenv())
if _:
    print("Environment Variables Loaded Successfully")
    print(os.getenv("API_24SEA_USERNAME"))
    # print(os.getenv("24SEA_API_PASSWORD"))
else:
    raise Exception("Environment Variables Not Loaded")


In [ ]:
# **DataFrame initialization**
# The empty DataFrame is created beforehand because it needs to authenticate
# with the API to fetch the data.
df = pd.DataFrame()


#### Metrics overview

The metrics overview table is a summary of the metrics that you can access through the 24SEA API. The table provides a brief description of each metric, the data type, unit of measurement, and some additional fields that can be used to filter the metrics.


In [ ]:
# **Metrics Overview**
# The metrics overview is a summary of the metrics available in the API.
m_o = df.datasignals._DataSignals__api.metrics_overview
m_o.head(10)


#### Data retrieval step


In [ ]:
# **Data Retrieval**

sites = ["belwind"]

locations = m_o[m_o["site"].str.lower() == "belwind"]["location"].unique().tolist()

metrics = ["cc c01", "dem"]

# start_timestamp = "2024-01-01T00:00:00Z"
# end_timestamp = "2025-01-01T01:00:00Z"
# See https://pypi.org/project/shorthand-datetime package for more information
# on how to use shorthand datetime strings to set start and end timestamps.
start_timestamp = "now-1Y/M"
end_timestamp = "now/M"

df.datasignals.get_data(
    sites, locations, metrics, start_timestamp, end_timestamp
)

In [ ]:
import rich
# Count percentage  of none in each column
none_perc_dict = {}
for col in df.columns:
    none_perc_dict[col] = f"{round(df[col].isna().sum() / len(df) * 100, 2)}%"
    
rich.print_json(data=none_perc_dict)


In [ ]:
none_perc_dict

In [ ]:
# Print one 10-minute cycle-count dictionary as an example
rich.print_json(df["CC_BB_C01_TP_SG_LAT019_Mtn"].iloc[0])

Convert 10-minute cycle-count dictionaries to `py-fatigue CycleCount` objects.

In [ ]:
df.fatigue.cycle_counts_to_objects()

In [ ]:
# For each column starting with CC_ in df, concatenate the stress_range and count_cycle arrays of every row
# into two very long arrays, and then plot the stress_range vs count_cycle as scatter plot.

cc_cols = [col for col in df.columns if col.startswith("CC_")]
from collections import defaultdict
# Initialize the defaultdict of dictionaries
cc_dict = defaultdict(lambda: defaultdict(list))
for col in cc_cols:
    cc_dict[col] = {
        'timestamp': [],
        'stress_range': [],
        'count_cycle': []
    }

print(cc_dict)

for idx, row in df.dropna.iterrows():
    for col in cc_cols:
        cc_cols[]
        cc_dict[col]['stress_range'].extend(row[col].stress_range)
        cc_dict[col]['count_cycle'].extend(row[col].count_cycle)


In [ ]:
# **Data Display**
if hasattr(pd.DataFrame, "to_markdown"):
    print(df.head().to_markdown())
else:
    print("Install tabulate package to use to_markdown()")

In [ ]:
dct = df.datasignals.as_dict()

In [ ]:
from api_24sea.core import to_star_schema

star_schema = to_star_schema(df, m_o)

In [ ]:
star_schema["FactData"]

In [ ]:
# Split the metric_id column by "_"
split_metric_id = star_schema['FactData']['metric_id'].str.split('_')

# Extract the necessary parts
part_0 = split_metric_id.str[0]
part_1 = split_metric_id.str[1:].str.join('_')

# Split the location column into location_short_id and site_short_id
star_schema['FactData']['location_short_id'] = star_schema['FactData']['location'].str[:2]
star_schema['FactData']['site_short_id'] = star_schema['FactData']['location'].str[2:]

# # Reformulate the metric_id column
star_schema['FactData']['metric_id_2'] = part_0 + '_' + star_schema['FactData']['location_short_id'] + '_' + star_schema['FactData']['site_short_id'] + '_' + part_1


In [ ]:
star_schema['FactData']